# Train DeepMN models across model configurations

Shubhayu Bhattacharyay
<br>
Ari Ercole

## I. Initialization

### Import necessary packages

In [2]:
# Fundamental methods
import os
import sys
import json
import time
import glob
import random
import warnings
import itertools
import numpy as np
import pandas as pd
import pickle as cp
import seaborn as sns
from scipy import stats
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
warnings.filterwarnings(action="ignore")

# Tensorflow, and CORAL methods (neural network methods)
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import tensorflow.python.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Activation, Dense, Dropout, Conv2D, Flatten, LSTM, Permute, Reshape, AlphaDropout, BatchNormalization
import coral_ordinal as coral

# Keras Tuner methods
import kerastuner as kt
from kerastuner.tuners import RandomSearch, Hyperband

# SciKit-Learn methods
from sklearn.metrics import accuracy_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelBinarizer, PowerTransformer, label_binarize
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, f1_score

# Import hiplot to visualize tuning results
import hiplot as hip

# Load custom functions
%run -i 'functions/ordinal_encoding.py'
%run -i 'functions/multiclass_metrics.py'

## II. Establishing learning environment for DeepMN models

### Define function to build and compile DeepMN functions of different hyperparameters

In [3]:
def build_deepMN(no_layers,units_vector):
    deepMN = Sequential()
    deepMN.add(Input(shape=(10,)))
    # Adding variable amount of hidden layers
    for i in range(no_layers):
        deepMN.add(Dense(
            units=units_vector[i],
            activation='relu'))
        deepMN.add(Dropout(0.2))
    # Adding final output layer     
    deepMN.add(Dense(7, activation='softmax'))
    # Compiling neural network model
    deepMN.compile(
        optimizer=tf.keras.optimizers.RMSprop(), 
        loss='categorical_crossentropy', 
        metrics=[tf.keras.metrics.CategoricalAccuracy(),f1_score_m])
    # Return neural network function
    return deepMN

### Define tuning grid for model configurations

In [3]:
layer_options = range(2,5)
units_options = [128,256,512]
smote_options = [0,1]

layer_smote_combos = np.array(np.meshgrid(layer_options, smote_options)).reshape(2, len(layer_options)*len(smote_options)).T

tuning_grid = pd.DataFrame(np.empty((0,3)))
for i in range(np.shape(layer_smote_combos)[0]):
    curr_tuning_options = pd.DataFrame({"layers":layer_smote_combos[i,0],\
                                        "smote": layer_smote_combos[i,1],\
                                        "units": list(itertools.product(units_options,repeat = layer_smote_combos[i,0]))})
    tuning_grid.columns = curr_tuning_options.columns
    tuning_grid = tuning_grid.append(curr_tuning_options,ignore_index=True)
tuning_grid["tune_idx"] = np.arange(tuning_grid.shape[0]) + 1
tuning_grid.to_csv('../repeated_cv/dl_tuning_grid.csv',index=False)

## III. Train DeepMN with Bootstrap Bias Corrected with Dropping Cross-Validation (BBCD-CV)

In [ ]:
# Establish dataframe of all tuning index-GOSE-class combinations for AUC
viable_indices = pd.DataFrame(list(itertools.product(tuning_grid.tune_idx.astype('int').unique(), ['1','3','4','5','6','7','8','macro-averaged'])),\
                              columns=['tune_idx','class'])

# Initialize empty dataframe to store indices that reject null hypothesis (and are thus banned from future interations)
banned_tuning_indices = pd.DataFrame(np.empty((0,3)),columns = ['class','aurocs','tune_idx'])

# Load testing folds
testing_folds = pd.read_csv('../testing_folds.csv')

# Initalize empty dataframe to store compiled prediction results
deepMN_results_compiled = pd.DataFrame(np.empty((0,16)))

# Inititalize dynamic display messages for status updates
dh = display('',display_id=True)
bs_message = display('',display_id=True)
bs_intermediate_status =  display('',display_id=True)
banned_message = display('',display_id=True)

# Iterate through repeat and fold directories
for curr_repeat in testing_folds['repeat.no'].unique():
    for curr_fold in testing_folds['fold.no'].unique():
        
        # Load normalized training set and encode for deepMN
        curr_norm_training_set = pd.read_csv(os.path.join('../repeated_cv','Repeat' + str(curr_repeat).zfill(2),'Fold' + str(curr_fold).zfill(1),'norm_train_dataframe.csv'))
        curr_norm_training_labels = label_binarize(curr_norm_training_set.GOSE.values,classes=[1,3,4,5,6,7,8])
        curr_norm_training_matrix = curr_norm_training_set.drop(columns=['entity_id','PatientType', 'GCS','GOSE']).values
        curr_norm_training_class_weights = dict(enumerate(compute_class_weight("balanced", np.sort(curr_norm_training_set.GOSE.unique()), curr_norm_training_set.GOSE.values)))
   
        # Load SMOTEd normalized training set and encode for deepMN
        curr_smote_norm_training_set = pd.read_csv(os.path.join('../repeated_cv','Repeat' + str(curr_repeat).zfill(2),'Fold' + str(curr_fold).zfill(1),'smote_norm_train_dataframe.csv'))
        curr_smote_norm_training_labels = label_binarize(curr_smote_norm_training_set.GOSE.values,classes=[1,3,4,5,6,7,8])
        curr_smote_norm_training_matrix = curr_smote_norm_training_set.drop(columns=['GOSE']).values
        curr_smote_norm_training_class_weights = dict(enumerate(compute_class_weight("balanced", np.sort(curr_smote_norm_training_set.GOSE.unique()), curr_smote_norm_training_set.GOSE.values)))

        # Load normalized testing set and encode for deepMN
        curr_norm_testing_set = pd.read_csv(os.path.join('../repeated_cv','Repeat' + str(curr_repeat).zfill(2),'Fold' + str(curr_fold).zfill(1),'norm_test_dataframe.csv'))
        curr_norm_testing_labels = label_binarize(curr_norm_testing_set.GOSE.values,classes=[1,3,4,5,6,7,8])
        curr_norm_testing_matrix = curr_norm_testing_set.drop(columns=['entity_id','PatientType', 'GCS','GOSE']).values
        
        # Make sub-directory for saving deepMN models in current fold and repeat combination
        os.makedirs(os.path.join('../repeated_cv','Repeat' + str(curr_repeat).zfill(2),'Fold' + str(curr_fold).zfill(1),'trained_models','deepMN'),exist_ok=True)
        
        # Loop through different viable tuning configurations (tune_idx) and train models
        for curr_tune_idx in (viable_indices.tune_idx.unique()):
            
            # Update dynamic message on current tuning status
            dh.update('tuning iteration ' + str(curr_tune_idx) + ' out of ' + \
                  str(len(viable_indices.tune_idx.unique())) + ' started for repeat: ' + str(curr_repeat)\
                  + ' and fold: ' + str(curr_fold))
            
            # Identify current model parameters based on tuning grid
            curr_layers = int(tuning_grid.layers[tuning_grid.tune_idx.astype('int') == curr_tune_idx].values)
            curr_units = tuning_grid.units[tuning_grid.tune_idx.astype('int') == curr_tune_idx].values[0]
            curr_smote = int(tuning_grid.smote[tuning_grid.tune_idx.astype('int') == curr_tune_idx].values[0])
            
            # Build and compile current model based on tuning grid parameters
            curr_mdl = build_deepMN(curr_layers,curr_units)
            
            # Select training data based on SMOTE indicator
            if curr_smote == 1:
                curr_mdl.fit(x=curr_smote_norm_training_matrix,\
                             y=curr_smote_norm_training_labels,\
                             batch_size=128,\
                             epochs=7,\
                             verbose = 0,\
                             validation_data = (curr_norm_testing_matrix,curr_norm_testing_labels),\
                             class_weight=curr_smote_norm_training_class_weights)        
            else:
                curr_mdl.fit(x=curr_norm_training_matrix,\
                             y=curr_norm_training_labels,\
                             batch_size=128,\
                             epochs=7,\
                             verbose = 0,\
                             validation_data = (curr_norm_testing_matrix,curr_norm_testing_labels),\
                             class_weight=curr_norm_training_class_weights)
            
            # Save current model in appropriate sub-directory
            curr_mdl.save(os.path.join('../repeated_cv','Repeat' + str(curr_repeat).zfill(2),\
                                       'Fold' + str(curr_fold).zfill(1),'trained_models','deepMN','deepMN_tuning_' + str(curr_tune_idx).zfill(3) + '.h5'))
            
            # Compile predictions from current tuning configuration and append to compiled prediction dataframe
            curr_mdl_test_pred_probs = pd.DataFrame(curr_mdl.predict(curr_norm_testing_matrix),columns=['prob_GOSE_1', 'prob_GOSE_2_3', 'prob_GOSE_4', 'prob_GOSE_5', 'prob_GOSE_6','prob_GOSE_7','prob_GOSE_8'])
            curr_mdl_test_pred_labels = pd.DataFrame(np.transpose(np.vstack((curr_norm_testing_set.GOSE.values,[[1,3,4,5,6,7,8][j] for j in curr_mdl.predict(curr_norm_testing_matrix).argmax(axis=-1)]))),columns=['true_labels','pred_labels'])
            curr_mdl_test_final = pd.concat([curr_mdl_test_pred_labels,curr_mdl_test_pred_probs],axis=1)
            curr_mdl_test_final["repeat.name"] = curr_repeat
            curr_mdl_test_final["fold.name"] = curr_fold
            curr_mdl_test_final["layers"] = int(curr_layers)
            curr_mdl_test_final["SMOTE"] = int(curr_smote)
            curr_mdl_test_final["units"] = [curr_units for j in range(curr_mdl_test_final.shape[0])]
            curr_mdl_test_final["tune_idx"] = int(curr_tune_idx)
            curr_mdl_test_final["entity_id"] = curr_norm_testing_set.entity_id
            deepMN_results_compiled.columns = curr_mdl_test_final.columns
            deepMN_results_compiled = deepMN_results_compiled.append(curr_mdl_test_final,ignore_index=True)
            
        ## BOOSTRAP-DROPPING:
        # Update bootstrap status message if more than one viable configuration remains
        if len(viable_indices.tune_idx.unique()) == 1:
            bs_message.update('only one viable configuration remaining. Bootstrapping skipped for repeat: ' + str(curr_repeat) + ' and fold: ' + str(curr_fold))
            continue
        else:
            bs_message.update('Bootstrapping started for repeat: ' + str(curr_repeat) + ' and fold: ' + str(curr_fold))
        
        # First, identify optimal configuration based on all available results (after each fold) for each GOSE class and macro-average
        aurocs_df = pd.DataFrame(np.empty((0,3)), columns = ['class','aurocs','tune_idx'])
        prob_cols = [col for col in deepMN_results_compiled if col.startswith('prob_GOSE_')]
        
        for curr_tune_idx in (viable_indices.tune_idx.unique()):
            # Filter out results with current tune index:
            curr_results_idx = list(itertools.compress(range(len(deepMN_results_compiled.tune_idx.astype('int') == int(curr_tune_idx))), deepMN_results_compiled.tune_idx.astype('int') == int(curr_tune_idx)))
            filt_results = deepMN_results_compiled.iloc[curr_results_idx,]
            curr_labels = label_binarize(filt_results.true_labels.astype('int').values,classes=[1,3,4,5,6,7,8])
            
            curr_aurocs = roc_auc_score(y_true = curr_labels, y_score = filt_results[prob_cols].values,average=None,multi_class='ovr')
            curr_aurocs_df = pd.DataFrame({'class':['1','3','4','5','6','7','8','macro-averaged'],\
                                           'aurocs':np.append(curr_aurocs,np.mean(curr_aurocs)),\
                                          'tune_idx':int(curr_tune_idx)})
            
            # Filter out classes that are still viable
            curr_still_viable_classes = viable_indices['class'][viable_indices.tune_idx == curr_tune_idx]
            curr_aurocs_df = curr_aurocs_df[curr_aurocs_df['class'].isin(curr_still_viable_classes)]
            
            # Append viable AUROC candidates to current pool
            aurocs_df = aurocs_df.append(curr_aurocs_df,ignore_index = True)
        
        # Identify optimal tuning index for each class and sort by class
        opt_aurocs_idx = aurocs_df.groupby(['class'])['aurocs'].transform(max) == aurocs_df['aurocs']
        opt_aurocs_df = aurocs_df[opt_aurocs_idx].sort_values(by=['class'])
        
        # Remove optimal rows from the pooled AUROC dataframe to determine which models will be tested for drouput
        other_aurocs_df = aurocs_df[~opt_aurocs_idx].sort_values(by=['class'])
        
        # Randomly draw sample-size-number of patients with replacement 200 times for boostrapping
        bootstrap_ids = np.random.choice(deepMN_results_compiled.entity_id.unique(),size = (len(deepMN_results_compiled.entity_id.unique()),200), replace = True)
        
        # Initialize indicator matrix to determine configurations to eliminate
        other_less_thans = np.empty([other_aurocs_df.shape[0],bootstrap_ids.shape[1]],dtype=int)
        
        # Loop through bootstrap indices
        for bs_idx in range(bootstrap_ids.shape[1]):
            # Signify current boostrapping index
            curr_bootstrap_ids = bootstrap_ids[:,bs_idx]
            
            # Filter out predictions of current bootstrap patients and tuning indices in the optimal dataframe
            opt_results_idx = (deepMN_results_compiled.entity_id.isin(curr_bootstrap_ids)) & (deepMN_results_compiled.tune_idx.astype('int').isin(opt_aurocs_df.tune_idx.astype('int').unique()))
            filt_results = deepMN_results_compiled[opt_results_idx]
            
            # Initialize empty dataframe to store 'best'-configuration AUROCs in the given boostrap sample
            curr_bs_opt_aurocs = pd.DataFrame(np.empty((0,3)), columns = ['class','aurocs','tune_idx'])
            
            for curr_opt_idx in opt_aurocs_df.tune_idx.astype('int').unique():
                
                #Filter out predictions and labels for current tuning configuration
                curr_opt_results_idx = (filt_results.tune_idx.astype('int') == curr_opt_idx)
                curr_opt_filt_results = filt_results[curr_opt_results_idx]
                curr_opt_labels = label_binarize(curr_opt_filt_results.true_labels.astype('int').values,classes=[1,3,4,5,6,7,8])
                
                # Calculate AUROCs for current optimal AUROC candidate and add approprate classes to current bootstrap optimals dataframe
                curr_opt_auroc = roc_auc_score(y_true = curr_opt_labels, y_score = curr_opt_filt_results[prob_cols].values,average=None,multi_class='ovr')
                curr_opt_aurocs_df = pd.DataFrame({'class':['1','3','4','5','6','7','8','macro-averaged'],\
                               'aurocs':np.append(curr_opt_auroc,np.mean(curr_opt_auroc)),\
                              'tune_idx':int(curr_opt_idx)})
                curr_opt_classes = opt_aurocs_df['class'][opt_aurocs_df.tune_idx == curr_opt_idx]
                curr_bs_opt_aurocs = curr_bs_opt_aurocs.append(curr_opt_aurocs_df[curr_opt_aurocs_df['class'].isin(curr_opt_classes)],ignore_index = True)
                
            # Sort current bootstrap optimals dataframe by GOSE class
            curr_bs_opt_aurocs = curr_bs_opt_aurocs.sort_values(by=['class'])
            
            # Iterate through other tuning indices and determine whether current bootstrap AUC is lower
            for curr_other_idx in other_aurocs_df.tune_idx.astype('int').unique():
                
                #Filter out predictions and labels for current suboptimal tuning configuration
                curr_other_results_idx = (deepMN_results_compiled.entity_id.isin(curr_bootstrap_ids))&(deepMN_results_compiled.tune_idx.astype('int') == curr_other_idx)
                curr_other_filt_results = deepMN_results_compiled[curr_other_results_idx]
                
                # Calculate AUROCs for current suboptimal AUROC candidate and filter out only viable classes and sort by class
                curr_other_labels = label_binarize(curr_other_filt_results.true_labels.astype('int').values,classes=[1,3,4,5,6,7,8])         
                curr_other_auroc = roc_auc_score(y_true = curr_other_labels, y_score = curr_other_filt_results[prob_cols].values,average=None,multi_class='ovr')
                curr_other_aurocs_df = pd.DataFrame({'class':['1','3','4','5','6','7','8','macro-averaged'],\
                                                   'aurocs':np.append(curr_other_auroc,np.mean(curr_other_auroc)),\
                                                   'tune_idx':int(curr_other_idx)})
                curr_other_classes = other_aurocs_df['class'][other_aurocs_df.tune_idx == curr_other_idx]
                curr_other_aurocs_df = curr_other_aurocs_df[curr_other_aurocs_df['class'].isin(curr_other_classes)].sort_values(by=['class'])

                # Identify indices of `other_aurocs_df` that match current 'curr_other_aurocs_df' rows
                indicator_idx = (other_aurocs_df.tune_idx.astype('int') == curr_other_idx) & (other_aurocs_df['class'].isin(curr_other_classes))
                
                # Compare current suboptimal AUCs to the optimal AUCs under current option
                other_less_thans[indicator_idx,bs_idx] = (curr_other_aurocs_df.aurocs.reset_index(drop=True) < curr_bs_opt_aurocs.aurocs[curr_bs_opt_aurocs['class'].isin(curr_other_classes)].reset_index(drop=True)).astype('int').values
            
            bs_intermediate_status.update('Bootstrap no. ' + str(bs_idx+1) + ' out of ' + str(bootstrap_ids.shape[1]) + ' completed.')
            
        # Add tuning indices that fail the hypothesis test for given GOSE classes to the running list of banned tuning indices
        banned_tuning_indices = banned_tuning_indices.append(other_aurocs_df[np.sum(other_less_thans,1)/bootstrap_ids.shape[1] >= .99],ignore_index = True)
        banned_tuning_class_combos = banned_tuning_indices[['tune_idx','class']]
        banned_tuning_class_combos.tune_idx = banned_tuning_class_combos.tune_idx.astype('int')
        
        # Update viable indices by removing banned tuning indices per class
        viable_indices = pd.concat([viable_indices, banned_tuning_class_combos, banned_tuning_class_combos]).drop_duplicates(keep=False)
        
        banned_message.update('Current number of banned indices: '+ str(banned_tuning_indices.shape[0])+' out of '+str(tuning_grid.shape[0]*8) +' possible config-class combinations \
        after repeat: ' + str(curr_repeat) + ' and fold: ' + str(curr_fold))
        
deepMN_results_compiled.to_csv('../repeated_cv/compiled_predictions/deepMN.csv',index=False)
banned_tuning_indices.to_csv('../repeated_cv/deepMN_banned_tuning_indices.csv',index = False)